In [223]:
import json
import numpy as np
import pandas as pd
import lxml
import re
import os
from datetime import datetime, date, time
import pyodbc
import requests

### Get EA script history in JSON format via the API

In [224]:
custom_headers  = {"Authorization": "Gt4qkl1ahFDcFyQwfjMBYT0utbLD4STHQjWdIXzsYH8v3w29M6Tm6tVKI0uMj6Qh", "Content-Type": "application/json"}
url = 'https://ea.executive.automats.app/api/bi/scripts?withHistory=true'

resp = requests.get(url, headers = custom_headers, verify=False)
print(resp.status_code)

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:999: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ea.executive.automats.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


200


In [225]:
json_data = json.loads(resp.text)
json_text = json.dumps(json_data, indent=4)

In [226]:
try:
    os.remove('data/execution/try.json')
except:
    print('File does not exist')

with open('data/execution/try.json', 'w') as fw:
    fw.write(json_text)

### Remove invalid characters from the inbound JSON data

In [227]:
try:
    os.remove('data/execution/script_history_fixed.json')
except:
    print('File does not exist')

#with open('data/execution/script_history.json', 'r') as fr:
with open('data/execution/try.json', 'r') as fr:    
    # reading line by line
    lines = fr.readlines()
    last_line = len(lines)
    print(len(lines))
    
    for line in lines:
        line = re.sub("\u0003", "", line)
        with open('data/execution/script_history_fixed.json', 'a') as fw:
            fw.write(line) 

# Previous runtime = 13.8s

2840165


### Parse the multilevel JSON into a CSV file

In [228]:
f = open('data/execution/script_history_fixed.json')
data = json.loads(f.read())

In [229]:
try:
    os.remove('data/execution/check_structure.csv')
except:
    print('File does not exist')
with open('data/execution/check_structure.csv', 'a') as fw:
    #line = 'topProjectName,name,sessionId,executionLogsId,scriptId,scriptName,script_status,label,step_state,step_status,startTime,endTime,duration,lineNum,StepLogsId,takeScreenshots,screenshotsOnlyOnInterruption\n'
    line = 'topProjectName,name,sessionId,executionLogsId,scriptId,scriptName,script_status,label,step_state,step_status,startTime,endTime,duration,lineNum,StepLogsId\n'
    fw.write(line)
    i = 0
    for level1 in data:
        for key, value in level1.items():
            #if i == 100: break
            i = i + 1
            #takeScreenshots_str = str(level1['takeScreenshots'])
            #screenshotsOnlyOnInterruption_str = str(level1['screenshotsOnlyOnInterruption'])            
            try:
                executionLogs_str = json.dumps(level1['executionLogs'])
                executionLogs_json = json.loads(executionLogs_str)
                for level2 in executionLogs_json:
                    for key2, value2 in level2.items():
                        #try:
                        executionStepLogs_str = json.dumps(level2['executionStepLogs'])
                        executionStepLogs_json = json.loads(executionStepLogs_str)
                        if key2 == 'executionStepLogs':
                            records_no_str = str(i)
                            topProjectName = str(level1['topProjectName'])
                            name_str = str(level1['name'])
                            # Remove any commas from name 
                            check = (',' in name_str)
                            if check == True:
                                name_str = re.sub(',', ';', name_str)                            
                            sessionId_str = str(level2['sessionId'])
                            id_str = str(level2['id'])
                            scriptId_str = str(level2['scriptId'])
                            scriptName_str = str(level2['scriptName'])
                            # Remove any commas from scriptName 
                            check = (',' in scriptName_str)
                            if check == True:
                                scriptName_str = re.sub(',', ';', scriptName_str)                             
                            state_str = str(level2['state'])
                            status2_str = str(level2['status'])
                            lastIssues_str = str(level2['lastIssues'])
                            # Remove any commas from lastIssues_str
                            check = (',' in lastIssues_str)
                            if check == True:
                                lastIssues_str = re.sub(',', ';', lastIssues_str)


                            for level3 in executionStepLogs_json:
                                #for key3, value3 in level3.items():
                                status3_str = str(level3['status'])
                                label_str = str(level3['label'])
                                
                                # These lines replace long text test step labels with short text (that does not wrap)
                                check = ('Click Dynamics 365' in label_str)
                                if check == True:
                                    label_str = 'Click Dynamics 365'
                                check = ('Click Sales Hub' in label_str)
                                if check == True:
                                    label_str = 'Click Sales Hub'
                                check = ('Type value function eaExecuteVariableValue()' in label_str)
                                if check == True:
                                    label_str = 'Type value function eaExecuteVariableValue()'
                                check = ('Select from function eaExecuteVariableValue()' in label_str)
                                if check == True:
                                    label_str = 'Select from function eaExecuteVariableValue()'                                    
                                check = ('Select lookup values function eaExecuteVariableValue()' in label_str)
                                if check == True:
                                    label_str = 'Select lookup values function eaExecuteVariableValue()'
                                # Remove any commas from label_str 
                                check = (',' in label_str)
                                if check == True:
                                    label_str = re.sub(',', ';', label_str)
                                # Now replace all newlines that remain in label_str
                                check = ('\n' in label_str)
                                if check == True:
                                    label_str = re.sub('\n', '/nl/', label_str)
                                # Done with replacing text

                                startTime_str = str(level3['startTime'])
                                endTime_str = str(level3['endTime'])
                                duration_str = str(level3['duration'])
                                lineNum_str = str(level3['lineNum'])
                                StepLogs_id_str = str(level3['id'])

                                #line = topProjectName+','+name_str+','+sessionId_str+','+id_str+','+scriptId_str+','+scriptName_str+','+status2_str+','+label_str+','+state_str+','+status3_str+','+startTime_str+','+endTime_str+','+duration_str+','+lineNum_str+','+StepLogs_id_str+','+takeScreenshots_str+','+screenshotsOnlyOnInterruption_str+'\n'
                                line = topProjectName+','+name_str+','+sessionId_str+','+id_str+','+scriptId_str+','+scriptName_str+','+status2_str+','+label_str+','+state_str+','+status3_str+','+startTime_str+','+endTime_str+','+duration_str+','+lineNum_str+','+StepLogs_id_str+'\n'
                                fw.write(line)
                            # except:
                            #     print('Exception on Level2 for Level1 item '+str(i))
            except:
                #print('Exception for Level1 item '+str(i))
                pass
    print('Processed '+str(i)+' items')           

    # Previous runtime = 6m 21s     


Processed 10641 items


### Read the denormalized CSV file into a Pandas dataframe

In [343]:
df = pd.read_csv('data/execution/check_structure.csv')

# Previous runtime = 6.6s 

/tmp/ipykernel_590/2474359986.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/execution/check_structure.csv')


In [344]:
df = df.drop_duplicates()
df = df.reset_index(drop=True)
df.rename(columns={'label':'StepLabel'}, inplace=True)
df.rename(columns={'name':'testName'}, inplace=True)

In [345]:
mask = (df['startTime'] == 'None') 
df['startTime'].mask(mask,'2022-01-01T00:00:00.000Z',inplace=True)
df['startTime'] = pd.to_datetime(df['startTime'])

In [346]:
mask = (df['endTime'] == 'None') 
df['endTime'].mask(mask,'2022-01-01T00:00:00.000Z',inplace=True)
df['endTime'] = pd.to_datetime(df['endTime'])

In [347]:
mask = (df['duration'] == 'None') 
df['duration'].mask(mask,'0',inplace=True)
df['duration'] = pd.to_numeric(df['duration'])

In [348]:
df['duration_sec'] = df.duration/1000
df['duration_mins'] = df.duration/1000/60

In [349]:
df['date'] = pd.to_datetime(df['startTime'], format='%Y-%m-%d').dt.date
df['date'] = pd.to_datetime(df['date'])


In [350]:
df['startTime'] = pd.to_datetime(df['startTime']).dt.time
df['endTime'] = pd.to_datetime(df['endTime']).dt.time

In [351]:
df['time'] = df['startTime'].astype(str)
df['time'] = df['time'].str[0:5]
df['time'] = df['time'].str.replace(':','.')
df['time'] = df['time'].astype(float)

0          9.07
1          9.07
2          9.07
3          9.05
4          9.05
          ...  
261951    11.49
261952    11.50
261953    11.50
261954    11.50
261955    11.51
Name: time, Length: 261956, dtype: float64

In [353]:
# Filter out all history from before the first run
df = df.query("date > '2023-01-18'").copy()

### Add a column to indicate Runs of each test script
As there is no other field available via the API, this field is generated each time the lineNum == 1.  This assumes that there will always be a step 1 in every test case.
Before this can be done, first sort the whole dataframe

In [354]:
df.sort_values(by=['executionLogsId','StepLogsId'], inplace=True)
df = df.reset_index(drop=True)

In [355]:
df['run'] = 0
run = 0
for idx in df.index:
    if df['lineNum'][idx] == 1:
        run = run + 1
    df['run'][idx] = run

 # Previous runtime = 14.8s    

/tmp/ipykernel_590/1847471764.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['run'][idx] = run


### Generate a field to indicate where test scripts have run to the last step of the script
This assumes that the last step is "End script"
First sort the dataframe by Runs
Then add a new text field that is a concatenation of all the step labels for each run
Finally, check which of those strings contain the text "End script"

In [356]:
runs_np = df.run.unique()
end_script_dict = {}
for i in runs_np:
    end_script_dict[i] = 'Script stopped'
step_error_dict = {}
for i in runs_np:
    step_error_dict[i] = 'No step errors'

In [357]:
df.sort_values(by=['run'], inplace=True)
df = df.reset_index(drop=True)

In [358]:
for idx in df.index:
    #if run == 10: break
    run = df['run'][idx]
    if df['StepLabel'][idx] == 'End script':
        end_script_dict[run] = 'Script completed'
    if df['step_status'][idx] == 'ERROR':
        step_error_dict[run] = 'One or more step errors'

In [359]:
end_script_df = pd.DataFrame.from_dict(end_script_dict,orient ='index',columns=['script_completion'])
end_script_df.reset_index(inplace=True)
end_script_df.rename(columns={'index':'run'}, inplace=True)

In [360]:
step_error_df = pd.DataFrame.from_dict(step_error_dict,orient ='index',columns=['step_error'])
step_error_df.reset_index(inplace=True)
step_error_df.rename(columns={'index':'run'}, inplace=True)

In [361]:
df = pd.merge(
    df,
    end_script_df,
    how="inner",
    on=None,
    left_on='run',
    right_on='run',
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

In [362]:
df = pd.merge(
    df,
    step_error_df,
    how="inner",
    on=None,
    left_on='run',
    right_on='run',
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

In [363]:
df.rename(columns={'script_status': 'EA_script_status','step_state': 'EA_step_state','step_status': 'EA_step_status','step_error': 'IPT_step_error','script_completion': 'IPT_script_completion'}, inplace=True)

In [364]:
df['KPI'] = 'Not mapped'

In [365]:
# K1
mask = (df['scriptName'].str.contains('Place a sales local order'))
df['KPI'].mask(mask,'K01 Place a sales local order',inplace=True)

# K2
mask = (df['scriptName'].str.contains('Place a B2B local order'))
df['KPI'].mask(mask,'K02 Place a B2B/EDI order and Send Confirmation',inplace=True)

# K3 Place Service Portal Order
# No test script

# K4
mask = (df['scriptName'].str.contains('Amend an existing sales order'))
df['KPI'].mask(mask,'K04 Amend an existing sales order',inplace=True)

# K5
mask = (df['scriptName'].str.contains('Trade Returns Order'))
df['KPI'].mask(mask,'K05 Place a Trade Returns Order',inplace=True)

# K6
mask = (df['scriptName'].str.contains('Submit a B2B Remittance'))
df['KPI'].mask(mask,'K06 B2B Remittance',inplace=True)

# K7 B2B Remittance Adjustment
# No test script

#K8
mask = (df['scriptName'].str.contains('Receipted payment processing'))
df['KPI'].mask(mask,'K08 Receipted payment processing SA Only at the moment',inplace=True)

# K9 aDSD Batch Job posting invoice for HHD billing document 
# No test script

# K10
mask = (df['scriptName'].str.contains('Credit status check'))
df['KPI'].mask(mask,'K10 Credit status check',inplace=True)

# K11
mask = (df['scriptName'].str.contains('Send Load to Roadnet'))
df['KPI'].mask(mask,'K11 Send Load to Roadnet load for Planning',inplace=True)

# K12
mask = (df['scriptName'].str.contains('Receive load from Roadnet'))
df['KPI'].mask(mask,'K12 Receive load from Roadnet into D365',inplace=True)

# K12b
mask = (df['scriptName'].str.contains('Release to warehouse') & df['scriptName'].str.contains('Roadnet loads'))
df['KPI'].mask(mask,'K12b Release to Warehouse (Roadnet loads)',inplace=True)

# K13
mask = ((df['scriptName'].str.contains('Release to Warehouse') | df['scriptName'].str.contains('Release to warehouse')) & df['scriptName'].str.contains('manual'))
df['KPI'].mask(mask,'K13 Release to Warehouse (manually planned loads)',inplace=True)

# K14 Complete Picking Work & Process OOS 
# Not mapped??

# K14b
mask = (df['scriptName'].str.contains('Complete Picking Work') & df['scriptName'].str.contains('manual loads'))
df['KPI'].mask(mask,'K14b Complete Picking Work & Process OOS (manually planned loads)',inplace=True)

# K15
mask = (df['scriptName'].str.contains('aDSD Load confirmation'))
df['KPI'].mask(mask,'K15 aDSD Load confirmation',inplace=True)

# K16
mask = (df['scriptName'].str.contains('Load upload') & df['scriptName'].str.contains('Settlement'))
df['KPI'].mask(mask,'K16 Load upload & Settlement',inplace=True)

# K17
mask = (df['scriptName'].str.contains('Create a cost estimate'))
df['KPI'].mask(mask,'K17 Create a cost estimate for all standard costed procured materials',inplace=True)

# K18
mask = (df['scriptName'].str.contains('Imported Statistical'))
df['KPI'].mask(mask,'K18 Import Actual Statistical entries, into Cost Accounting',inplace=True)

# K19
mask = (df['scriptName'].str.contains('Distribute the range of items pending prices'))
df['KPI'].mask(mask,'K19 Distribute the range of items pending prices to another site',inplace=True)

# K20
mask = ((df['scriptName'].str.contains('Cost Rollup') & df['scriptName'].str.contains('Cost Allocation')) | df['scriptName'].str.contains('Maintain cost distribution table'))
df['KPI'].mask(mask,'K20 Run Cost Rollup and Cost Allocation Policies',inplace=True)

# K21
mask = (df['scriptName'].str.contains('Place SFA order'))
df['KPI'].mask(mask,'K21 Place SFA order and Send Confirmation',inplace=True)

In [366]:
df.sort_values(by=['run','lineNum','startTime'], inplace=True)
df = df.reset_index(drop=False)
df.rename(columns={'index': 'ISN'}, inplace=True)

In [184]:
raise SystemExit("Stop right here!")

SystemExit: Stop right here!

/home/wo/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Filter the dataset for output on a specific test cycle

In [381]:
exclude_scripts = ['TEST TIMINGS','Test - 10 sessions', 'Gary Test']

In [382]:
df1 = df.query("date == '2023-01-30' and time > 12.30 and testName not in @exclude_scripts")  # Remember time is UTC

In [383]:
# Alternative way to filter for time

# mask = ((df.date == '2023-01-30') & (df.startTime.dt.time > time(14,30)))  # Query does not work with the second part of this boolean expression
# df1 = df[mask]
# df1 = df1.query("testName not in @exclude_scripts")  # This does not work in the filter above, so apply them seperately

In [384]:
df1.to_csv('data/execution/results_all.csv',index=False)

In [385]:
df2 = df.query("date == '2023-01-30' and time > 12.30 and testName not in @exclude_scripts and IPT_script_completion != 'Script stopped' and IPT_step_error != 'One or more step errors'").copy()

In [386]:
df2 = df2.drop_duplicates(keep='first')
df2 = df2.reset_index(drop=True)

In [387]:
z = df2.groupby(['testName','run']).agg({'scriptId': {'first'}, 'duration_sec': {'sum'}}).reset_index()

In [388]:
z.sort_values(by=['testName'], inplace=True)
z = z.reset_index(drop=True)
z.columns = ['_'.join(col) for col in z.columns]

In [389]:
z.rename(columns={'testName_': 'testName', 'run_': 'run', 'scriptId_first': 'scriptId', 'duration_sec_sum': 'duration_sec'}, inplace=True)

In [390]:
z['90percQ'] = 0

In [391]:
scriptID_np = z['scriptId'].unique()

In [392]:
def calculate_outliers(x, scr_id):
    mask = (x['scriptId'] == scr_id)
    y = x[mask]
    q09= y.quantile(.9, numeric_only=True)
    #print(q09['duration_sec_sum'])
    x['90percQ'].mask(mask,q09['duration_sec'],inplace=True)

In [393]:
for scriptId in scriptID_np:
    calculate_outliers(z,scriptId)

In [394]:
mask = (z['duration_sec'] <= z['90percQ'])
z[mask]

,testName,run,scriptId,duration_sec,90percQ
0,10.04.09.07 Create a cost estimate for all sta...,5177,59,114.898,114.8980
1,Copy 1 - Receipted payment processing : SA Onl...,5520,414,123.926,123.9260
2,Copy 10 - Receipted payment processing : SA On...,5504,423,127.669,127.6690
3,Copy 12 - Receipted payment processing : SA On...,5498,425,132.997,132.9970
4,Copy 13 - Receipted payment processing : SA On...,5494,426,133.768,133.7680
...,...,...,...,...,...
257,[CE} 08.03.01.02 Place a Trade Returns Order,5387,100,1771.447,2860.4825
258,[CE} 08.03.01.02 Place a Trade Returns Order,5384,100,1898.790,2860.4825
259,[CE} 08.03.01.02 Place a Trade Returns Order,5379,100,1470.723,2860.4825
260,[CE} 08.03.01.02 Place a Trade Returns Order,5403,100,1499.655,2860.4825


In [395]:
z1 = df2.groupby(['KPI','run']).agg({'scriptId': {'first'}, 'duration_sec': {'sum'}}).reset_index()
z1.sort_values(by=['KPI'], inplace=True)
z1 = z1.reset_index(drop=True)
z1.columns = ['_'.join(col) for col in z1.columns]
z1.rename(columns={'KPI_': 'KPI', 'run_': 'run', 'scriptId_first': 'scriptId', 'duration_sec_sum': 'duration_sec'}, inplace=True)
z1['90percQ'] = 0
scriptID_np = z1['scriptId'].unique()
def calculate_outliers(x, scr_id):
    mask = (x['scriptId'] == scr_id)
    y = x[mask]
    q09= y.quantile(.9, numeric_only=True)
    #print(q09['duration_sec_sum'])
    x['90percQ'].mask(mask,q09['duration_sec'],inplace=True)
for scriptId in scriptID_np:
    calculate_outliers(z1,scriptId)

In [396]:
#z1.groupby(['KPI']).agg({'duration_sec': {'mean'}, 'run': {'count'}}).reset_index().sort_values(by=['KPI']).to_csv('data/execution/KPIs.csv',index=False)
z1.groupby(['KPI']).agg({'duration_sec': {'mean'}, 'run': {'count'}}).reset_index().sort_values(by=['KPI'])

,KPI,duration_sec,run
,,mean,count
0,K01 Place a sales local order,1342.914429,28
1,K02 Place a B2B/EDI order and Send Confirmation,3821.971196,92
2,K05 Place a Trade Returns Order,1792.868944,18
3,K08 Receipted payment processing SA Only at th...,130.087667,18
4,K12b Release to Warehouse (Roadnet loads),3480.535833,6
5,K16 Load upload & Settlement,23.566000,1
6,K17 Create a cost estimate for all standard co...,114.898000,1
7,"K18 Import Actual Statistical entries, into Co...",146.826000,1
8,K20 Run Cost Rollup and Cost Allocation Policies,111.768000,2


### Generate an output file in CSV format

In [397]:
df2.to_csv('data/execution/results_passed.csv',index=False)

### Aggregate to extract measures

In [398]:
def summary(x):
    runs = x['executionLogsId'].nunique()
    runs = float(runs)
    duration = x['duration'].sum()/1000
    duration_90 = x['duration'].quantile(0.9).sum()/1000
    mean_s = x['duration'].mean()/1000
    mean_90 = x['duration'].quantile(0.9).mean()/1000
    count = duration/mean_s
    result = {
        #'aggr_duration': duration,
        #'aggr_duration_minutes': duration/60,
        #'runs': runs,
        'mean': mean_s,
        'mean_90': mean_90,
        'mean_minutes': duration/runs/60
        #'steps': count
    }
    return pd.Series(result).round(1)

#df2.groupby(['scriptName','sessionId']).apply(summary).to_html('data/execution/result.html')
df2.groupby(['testName', 'executionLogsId']).apply(summary).to_html('data/execution/result.html')
#df2.groupby(['testName', 'executionLogsId']).quantile(.90).apply(summary).to_html('data/execution/result.html')
#df2.groupby(['testName', 'executionLogsId']).apply(summary).to_excel('data/execution/result_groupby.xlsx')
#df2.groupby(['testName', 'executionLogsId']).agg({'executionLogsId': {'nunique'}, 'duration' : {'sum'}})
#df2.groupby(['testName']).apply(summary).to_html('data/execution/result.html')

### From here on the script is disabled.

In [399]:
# Insert this to prevent the cells below from trying to write to the sql table
raise SystemExit("Stop right here!")

SystemExit: Stop right here!

/home/wo/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Get ready to export to SQL

In [ ]:
mask = (df['date'] > '2023-01-10')
#df.loc[(df['date'] > '2023-01-10')]
df = df[mask]

In [ ]:
# https://learn.microsoft.com/en-us/sql/machine-learning/data-exploration/python-dataframe-pandas?view=sql-server-ver16
# Some other example server values are
# server = 'localhost\sqlexpress' # for a named instance
# server = 'myserver,port' # to specify an alternate port
server = 'tcp:csazniptlsql01.database.windows.net,1433' 
database = 'CSAZN-INT-IPT-L-DSQ-01' 
username = 'ccbaadmin' 
password = '@FireBase123' 
# ENCRYPT defaults to yes starting in ODBC Driver 18. It's good to always specify ENCRYPT=yes on the client side to avoid MITM attacks.
cnxn = pyodbc.connect('DRIVER={ODBC Driver 18 for SQL Server};SERVER='+server+';DATABASE='+database+';ENCRYPT=yes;UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [ ]:
# cursor = cnxn.cursor()

# cursor.execute('''
# 		CREATE TABLE ea_script_history (
# 			records_no int,
#             topProjectName nvarchar(255),
#             testName nvarchar(255),
#             sessionId nvarchar(255),
#             id int,
#             scriptId int,
#             scriptName nvarchar(255),
#             state nvarchar(255),
#             status nvarchar(255),
#             StepLabel nvarchar(255),
#             startTime datetime,
#             endTime datetime,
#             duration int,
#             lineNum int,
#             StepLogs_id int
# 			)
#                ''')

# cnxn.commit()

In [ ]:
cnxn = pyodbc.connect('DRIVER={ODBC Driver 18 for SQL Server};SERVER='+server+';DATABASE='+database+';ENCRYPT=yes;UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
# Insert Dataframe into SQL Server:
i = 0
for index, row in df.iterrows():
     cursor.execute("INSERT INTO ea_script_history (records_no,topProjectName,testName,sessionId,id,scriptId,scriptName,state,status,StepLabel,startTime,endTime,duration,lineNum,StepLogs_id) values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", row.records_no, row.topProjectName, row.testName, row.sessionId, row.id, row.scriptId, row.scriptName, row.state, row.status, row.StepLabel, row.startTime, row.endTime, row.duration, row.lineNum, row.StepLogs_id)
     i = i + 1
     if i == 1000:
          i = 0
          cnxn.commit()          
cnxn.commit()
cursor.close()

In [ ]:
# cnxn = pyodbc.connect('DRIVER={ODBC Driver 18 for SQL Server};SERVER='+server+';DATABASE='+database+';ENCRYPT=yes;UID='+username+';PWD='+ password)
# cursor = cnxn.cursor()
# # Insert Dataframe into SQL Server:
# for index, row in df.iterrows():
#      cursor.execute("INSERT INTO ea_script_history (startTime) values(?)", row.startTime)
# cnxn.commit()
# cursor.close()

In [ ]:
# Delete all the rows in the SQl script history table
cursor = cnxn.cursor()
cursor.execute("DELETE FROM ea_script_history")
cnxn.commit()
cursor.close()

In [ ]:
# Drop the SQl script history table (it must be re-created)
cursor = cnxn.cursor()
cursor.execute("DROP Table ea_script_history")
#cursor.execute("DROP Table temp_table")
cnxn.commit()
cursor.close()

In [ ]:
a = [12, 15]

In [ ]:
print(type(a))

In [ ]:
15 in a

### Mark those Test Script Steps that must be included in the time measurement

In [ ]:
testStepstoMeasure = {90: [12, 15], 100: [14]}

In [ ]:
df1['Include_in_measure'] = False

In [ ]:
i = 0
for idx in df1.index:
    i = i + 1
    #if i == 1000: break
    scriptid = df1['scriptId'][idx]
    try:
        linenum = df['lineNum'][idx]
        if linenum in testStepstoMeasure[scriptid]:
            # print(b[scriptid])
            df1['Include_in_measure'] = True
    except:
        pass

### Mark Test Runs where the test outcome must be overridden to Include them in the test results

In [ ]:
df1['Override failed status'] = False

In [ ]:
testRunstoOverride = [879,880]

In [ ]:
i = 0
for idx in df1.index:
    i = i + 1
    #if i == 1000: break
    runid = df1['run'][idx]
    try:
        if runid in testRunstoOverride:
            df1['Override failed status'] = True
    except:
        pass

In [ ]:
df1['scriptId'].unique()